#### harvestor/reservoir estimate of things/work



Our the gameplay for harvestor/reservoir would be like this

**input**
- number of parcels
- dimension/type of each parcels
- survey result of each parcels
- total time user wants to play

**output**
- number of harvestors of each parcel with their level, and type
- number of reservoirs of each parcel with their level, and type


Note that -
- We dont need gotchi info for harvestor/reservoir optimization
- hence we dont need the 2nd gameplay which we did in altaar where we tried to spread out gotchis across parcels
- earlier gotchis kinship increasing everyday increased gameplay complexity and runtime. hence for our current function, we might be able to do bulk computations often which might make it faster
- assuming here that, we dont need to optimize for reservoir. we can simply choose reservoir based on what is the harvestor. need to discuss, if it is a fair assumption
- another assumption we would be taking is that, the user hasn't built anything, he is starting from level 0

In [1]:
"""

optimizer for what harvestor to build and how many harvestor to build
I think what reservoir to build would be a simple function which can be determined by what harvestor are we building
which putting harvetor in optimizing function, remember to include reservoir as well, because they go hand in hand
so an important task would be to compute ideal relation between harvestor and reservoir

note that harvestor reservoir do not depend on number of gotchis (altaar depended on number of gotchis)
harvestor/reservoir simply depends on cost and money. so here we dont need 2 gameplays. just 1 gameplay would suffice
which is optimal gameplay.

another question to ponder is that. which harvestor should we construct i.e. fud, fomo, alpha or kek ?? how do we decide this ?


because the gotchi and gotchi level doesnt depend here. we are do lots of calculations at once
no need to run for loop for days, as we did in the altaar version. that would make computation faster



calculate roi for creating each of the 4 harvestors with different levels on each of different parcels
roi computation should also be done regarding how many harvestorss for each parcels
simply sort them by roi in decreasing order and build them and choose one by one if the money in that token is still present


example -

parcel_1
1 fud harvestor level 1 ROI --> x11
1 fud harvestor level 2 ROI --> x12
2 fud harvestor level1,1 ROI --> x211
2 fud harvestor level1,2 ROI --> x212
.
.
1 kek harvestor ROI -->
.
.
.


there can be 2 kinds of strategies, maximum yield and maximum ROI(max profit)
for now focus on maximum ROI one


question to ponder --> why cant we do roi kind of thing for altaar levels



# need info on spillover rate of harvestor levels





"""

'\n\noptimizer for what harvestor to build and how many harvestor to build\nI think what reservoir to build would be a simple function which can be determined by what harvestor are we building\nwhich putting harvetor in optimizing function, remember to include reservoir as well, because they go hand in hand\nso an important task would be to compute ideal relation between harvestor and reservoir\n\nnote that harvestor reservoir do not depend on number of gotchis (altaar depended on number of gotchis)\nharvestor/reservoir simply depends on cost and money. so here we dont need 2 gameplays. just 1 gameplay would suffice\nwhich is optimal gameplay.\n\nanother question to ponder is that. which harvestor should we construct i.e. fud, fomo, alpha or kek ?? how do we decide this ?\n\n\nbecause the gotchi and gotchi level doesnt depend here. we are do lots of calculations at once\nno need to run for loop for days, as we did in the altaar version. that would make computation faster\n\n\n\ncalcula

In [2]:
"""
there is a problem with ROI, instead we should work on Net Profit
"""

'\nthere is a problem with ROI, instead we should work on Net Profit\n'

In [3]:
import math
import pandas as pd
import seaborn as sns
import numpy as np
import random
import matplotlib.pyplot as plt
import requests
import json

#### reading all the harvestors

In [4]:
fud_harvester = pd.read_csv("data/fud_harvester.csv")
fomo_harvester = pd.read_csv("data/fomo_harvester.csv")
alpha_harvester = pd.read_csv("data/alpha_harvester.csv")
kek_harvester = pd.read_csv("data/kek_harvester.csv")

In [5]:
# fud_harvester = fud_harvester.replace(',','', regex=True)
# fomo_harvester = fomo_harvester.replace(',','', regex=True).astype(int)
# alpha_harvester = alpha_harvester.replace(',','', regex=True).astype(int)
# kek_harvester = kek_harvester.replace(',','', regex=True).astype(int)

In [6]:
kek_harvester.head(10)

,Level,FUD,FOMO,ALPHA,KEK,HarvestRate/Day,Built Time (Blocks)
0,1,115,73,0,3.1,0.30,0
1,2,163,104,0,3.7,0.71,101376
2,3,210,131,0,5.2,1.60,200084
3,4,221,188,0,15.0,2.70,304128
4,5,335,279,0,22.0,4.10,414720
5,6,514,429,0,34.0,7.00,506880
6,7,511,641,0,77.0,13.00,604800
7,8,688,861,0,103.0,24.00,698880
8,9,422,1055,0,169.0,45.00,811008


In [7]:
fud_harvester.columns

Index(['Level', 'FUD', 'FOMO', 'ALPHA', 'KEK', 'HarvestRate/Day',
       'Built Time (Blocks) '],
      dtype='object')

In [8]:
fud_harvester['Build Time (Days)'] = fud_harvester['Built Time (Blocks) ']/40000
fomo_harvester['Build Time (Days)'] = fomo_harvester['Built Time (Blocks)']/40000
alpha_harvester['Build Time (Days)'] = alpha_harvester['Built Time (Blocks)']/40000
kek_harvester['Build Time (Days)'] = kek_harvester['Built Time (Blocks)']/40000

#### reading all the reservoirs

In [9]:
fud_reservoir = pd.read_csv("data/fud_reservoir.csv")
fomo_reservoir = pd.read_csv("data/fomo_reservoir.csv")
alpha_reservoir = pd.read_csv("data/alpha_reservoir.csv")
kek_reservoir = pd.read_csv("data/kek_reservoir.csv")

In [10]:
# fud_reservoir = fud_reservoir.replace(',','', regex=True).astype(int)
# fomo_reservoir = fomo_reservoir.replace(',','', regex=True).astype(int)
# alpha_reservoir = alpha_reservoir.replace(',','', regex=True).astype(int)
# kek_reservoir = kek_reservoir.replace(',','', regex=True).astype(int)

In [11]:
fud_reservoir['Build Time (Days)'] = fud_reservoir['Built Time (Blocks)']/40000
fomo_reservoir['Build Time (Days)'] = fomo_reservoir['Built Time (Blocks)']/40000
alpha_reservoir['Build Time (Days)'] = alpha_reservoir['Built Time (Blocks)']/40000
kek_reservoir['Build Time (Days)'] = kek_reservoir['Built Time (Blocks)']/40000

In [12]:
alpha_reservoir.columns

Index(['Level', 'FUD', 'FOMO', 'ALPHA', 'KEK', 'Capacity',
       'Built Time (Blocks)', 'Build Time (Days)'],
      dtype='object')

#### reading other data

In [13]:
avg_alchemica_df = pd.read_csv("data/avg_parcel_yield.csv")
avg_alchemica_df = avg_alchemica_df.replace(',','', regex=True)
avg_alchemica_df.columns = ['ParcelType', 'FUD', 'FOMO', 'ALPHA', 'KEK']

cols = ['FUD', 'FOMO', 'ALPHA', 'KEK']
avg_alchemica_df[cols] = avg_alchemica_df[cols].apply(pd.to_numeric, errors='coerce', axis=1)

In [14]:
avg_alchemica_df.head(4)

,ParcelType,FUD,FOMO,ALPHA,KEK
0,Humble (Small),7077,3539,1769,708
1,Reasonable (Medium),28308,14154,7077,2831
2,Spacious (Large),226465,113233,56616,22647


In [15]:
avg_alchemica_per_parcel_dict = {
    'humble' : {
        'FUD': 7077, 'FOMO': 3539, 'ALPHA': 1769, 'KEK': 708
    },
    'reasonable': {
        'FUD': 28308, 'FOMO': 14154, 'ALPHA': 7077, 'KEK': 2831
    },
    'spacious': {
        'FUD': 226465, 'FOMO': 113233, 'ALPHA': 56616, 'KEK': 22647
    }
}

#### pre computations

In [20]:
def compute_cost_dict_for_harvestor_reservoirs(df):
    return { df['Level'][i]:
                          fud_factor*int(df['FUD'][i])+
                          fomo_factor*int(df['FOMO'][i])+
                          alpha_factor*int(df['ALPHA'][i])+
                          kek_factor*int(df['KEK'][i]) 
                          for i in range(df.shape[0]) }

In [21]:
fud_harvester_cost_dict = compute_cost_dict_for_harvestor_reservoirs(fud_harvester)
fomo_harvester_cost_dict = compute_cost_dict_for_harvestor_reservoirs(fomo_harvester)
alpha_harvester_cost_dict = compute_cost_dict_for_harvestor_reservoirs(alpha_harvester)
kek_harvester_cost_dict = compute_cost_dict_for_harvestor_reservoirs(kek_harvester)

fud_reservoir_cost_dict = compute_cost_dict_for_harvestor_reservoirs(fud_reservoir)
fomo_reservoir_cost_dict = compute_cost_dict_for_harvestor_reservoirs(fomo_reservoir)
alpha_reservoir_cost_dict = compute_cost_dict_for_harvestor_reservoirs(alpha_reservoir)
kek_reservoir_cost_dict = compute_cost_dict_for_harvestor_reservoirs(kek_reservoir)

In [22]:
#alpha_harvester_cost_dict

In [23]:
#alpha_reservoir_cost_dict

In [24]:
def compute_harvestor_harvestrate_dict(df):
    return {
        df['Level'][i]:df['HarvestRate/Day'][i]
        for i in range(9)
    }

In [25]:
fud_harvester_harvestrate = compute_harvestor_harvestrate_dict(fud_harvester)
fomo_harvester_harvestrate = compute_harvestor_harvestrate_dict(fomo_harvester)
alpha_harvester_harvestrate = compute_harvestor_harvestrate_dict(alpha_harvester)
kek_harvester_harvestrate = compute_harvestor_harvestrate_dict(kek_harvester)

In [26]:
def compute_harvestor_reservoir_build_time_dict(df):
    return {
        df['Level'][i]:df['Build Time (Days)'][i]
        for i in range(9)
    }

In [27]:
fud_harvester_build_time = compute_harvestor_reservoir_build_time_dict(fud_harvester)
fomo_harvester_build_time = compute_harvestor_reservoir_build_time_dict(fomo_harvester)
alpha_harvester_build_time = compute_harvestor_reservoir_build_time_dict(alpha_harvester)
kek_harvester_build_time = compute_harvestor_reservoir_build_time_dict(kek_harvester)

fud_reservoir_build_time = compute_harvestor_reservoir_build_time_dict(fud_reservoir)
fomo_reservoir_build_time = compute_harvestor_reservoir_build_time_dict(fomo_reservoir)
alpha_reservoir_build_time = compute_harvestor_reservoir_build_time_dict(alpha_reservoir)
kek_reservoir_build_time = compute_harvestor_reservoir_build_time_dict(kek_reservoir)

In [28]:
def compute_reservoir_capacity_dict(df):
    return {
        df['Level'][i]:df['Capacity'][i]
        for i in range(9)
    }

In [29]:
fud_reservoir_capacity = compute_reservoir_capacity_dict(fud_reservoir)
fomo_reservoir_capacity = compute_reservoir_capacity_dict(fomo_reservoir)
alpha_reservoir_capacity = compute_reservoir_capacity_dict(alpha_reservoir)
kek_reservoir_capacity = compute_reservoir_capacity_dict(kek_reservoir)

In [30]:
def harvestor_cost_finder(x):
    return {
        'FUD': fud_harvester_cost_dict,
        'FOMO': fomo_harvester_cost_dict,
        'ALPHA': alpha_harvester_cost_dict,
        'KEK': kek_harvester_cost_dict
    }.get(x)

In [31]:
def reservoir_cost_finder(x):
    return {
        'FUD': fud_reservoir_cost_dict,
        'FOMO': fomo_reservoir_cost_dict,
        'ALPHA': alpha_reservoir_cost_dict,
        'KEK': kek_reservoir_cost_dict
    }.get(x)

In [32]:
def harvestor_harvestrate_finder(x):
    return {
        'FUD': fud_harvester_harvestrate,
        'FOMO': fomo_harvester_harvestrate,
        'ALPHA': alpha_harvester_harvestrate,
        'KEK': kek_harvester_harvestrate
    }.get(x)

In [33]:
def reservoir_capacity_finder(x):
    return {
        'FUD': fud_reservoir_capacity,
        'FOMO': fomo_reservoir_capacity,
        'ALPHA': alpha_reservoir_capacity,
        'KEK': kek_reservoir_capacity
    }.get(x)

In [34]:
def harvestor_build_time_finder(x):
    return {
        'FUD': fud_harvester_build_time,
        'FOMO': fomo_harvester_build_time,
        'ALPHA': alpha_harvester_build_time,
        'KEK': kek_harvester_build_time
    }.get(x)

In [35]:
def reservoir_build_time_finder(x):
    return {
        'FUD': fud_reservoir_build_time,
        'FOMO': fomo_reservoir_build_time,
        'ALPHA': alpha_reservoir_build_time,
        'KEK': kek_reservoir_build_time
    }.get(x)

In [36]:
class HelperDictsClass:
    
    reservoir_cost_dict = None
    reservoir_capacity = None
    reservoir_build_time = None
    harvester_cost_dict = None
    harvester_build_time = None
    harvester_rate = None
    
    
    def __init__(self, alchemica_type):
        self.reservoir_cost_dict = reservoir_cost_finder(alchemica_type)
        self.reservoir_capacity = reservoir_capacity_finder(alchemica_type)
        self.reservoir_build_time = reservoir_build_time_finder(alchemica_type)
        self.harvester_cost_dict = harvestor_cost_finder(alchemica_type)
        self.harvester_build_time = harvestor_build_time_finder(alchemica_type)
        self.harvester_rate = harvestor_harvestrate_finder(alchemica_type)



In [37]:
helper_dicts = {
    'FUD': HelperDictsClass('FUD'),
    'FOMO': HelperDictsClass('FOMO'),
    'ALPHA': HelperDictsClass('ALPHA'),
    'KEK': HelperDictsClass('KEK')
}

In [38]:
# harvestor_cost_dict = harvestor_cost_finder(alchemica_type)
# harvester_build_time = harvestor_build_time_finder(alchemica_type)
# harvester_rate = harvestor_harvestrate_finder(alchemica_type)

# reservoir_cost_dict = reservoir_cost_finder(alchemica_type)
# reservoir_build_time = reservoir_build_time_finder(alchemica_type)

#### defining other constants

In [16]:
daily_reservoir_empty_frequency = 3

In [17]:
fud_factor = 1.0
fomo_factor = 2.0
alpha_factor = 4.0
kek_factor = 10.0

try:
    URL = "https://api.coingecko.com/api/v3/simple/price?ids=aavegotchi-fud%2Caavegotchi-fomo%2Caavegotchi-alpha%2Caavegotchi-kek%2Caavegotchi&vs_currencies=usd"
    result = requests.get(url = URL).json()

    fud_factor = 1.0
    fomo_factor = result['aavegotchi-fomo']['usd']/result['aavegotchi-fud']['usd']
    alpha_factor = result['aavegotchi-alpha']['usd']/result['aavegotchi-fud']['usd']
    kek_factor = result['aavegotchi-kek']['usd']/result['aavegotchi-fud']['usd']
except:
    print("unable to get token factors from the API")
    

In [18]:
alchemica_factor_dict = {
    'FUD': fud_factor,
    'FOMO': fomo_factor,
    'ALPHA': alpha_factor,
    'KEK': kek_factor
}
alchemica_factor_dict

{'FUD': 1.0,
 'FOMO': 2.7480987838280755,
 'ALPHA': 4.021756237158579,
 'KEK': 7.87385771813631}

In [19]:
#select which parcel to build on
#select which harvestor to build on

In [ ]:
# we know total time T
# we know how much we want to extract from it
# we know how much harvest per day is need to extract it fully in given time T
# 

Question: for a given time T and given harvestor level, what reservoir would you build ?

In [ ]:
# assuming that the reservoir can be emptied only every 8 hours. 
# that means the total capacity of the reservoir for the day would be 3 times

# this wont depend on the given time T, as we just need to ensure the minimum harvestor for that level each day

In [ ]:
# doing computation only when 1 harvestor is allowed per parcel
T = 200
num_fud = 1000


In [ ]:
#fud_harvester.head(10)

In [ ]:
fud_reservoir.head(10)

In [ ]:
fud_harvester.columns

In [ ]:
fud_reservoir.columns

In [ ]:
fud_harvester_rate = fud_harvester_harvestrate

In [ ]:
reservoir_for_harvester_dict = {}

In [ ]:
reservoir_for_harvester_dict = {}

for harvestor_level in range(1, 10):
    for reservoir_level in range(1,10):
        if daily_reservoir_empty_frequency*fud_reservoir_capacity[reservoir_level] >= fud_harvester_rate[harvestor_level]:
            reservoir_for_harvester_dict[harvestor_level] = reservoir_level
            break


In [ ]:
reservoir_for_harvester_dict

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))         # Sample figsize in inches
#plt.bar(indices_list, res)
sns.barplot(list(reservoir_for_harvester_dict.keys()), list(reservoir_for_harvester_dict.values()))
plt.title('reservoir level for given harvestor level', fontsize = 20) # title with fontsize 20
plt.xlabel('harvestor level', fontsize = 12) # x-axis label with fontsize 15
plt.ylabel('reservoir level', fontsize = 12) # y-axis label with fontsize 15

question: is the upgrading process of reservoir faster or slower than that of harvestor

Question: for a given time T, what harvestor will you build?

In [ ]:
fud_survey_result = 10000
T = 200

In [ ]:
# how much can I harvest if I just build till level i for a given time T

In [ ]:
fud_harvester_build_time

In [ ]:
fud_harvester_rate

In [ ]:
result = {}

for i in range(1, 10):
    
    harvest_till_now = 0
    days_till_now = fud_harvester_build_time[1]
    
    for j in range(1, i):
        
        harvest_till_now += fud_harvester_rate[j]*fud_harvester_build_time[j+1]
        days_till_now += fud_harvester_build_time[j+1]
    
    result[i] = harvest_till_now + (T-days_till_now) * fud_harvester_rate[i]


In [ ]:
result

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))         # Sample figsize in inches
#plt.bar(indices_list, res)
sns.barplot(list(result.keys()), list(result.values()))
plt.title('yield when only upgraded till level i', fontsize = 20) # title with fontsize 20
plt.xlabel('harvestor level', fontsize = 12) # x-axis label with fontsize 15
plt.ylabel('yield', fontsize = 12) # y-axis label with fontsize 15

Question: for a given time T, and given survey value of alchemica, and given harvestor level, what reservoir would you build ?

Question analyse marginal gains in upgrading

In [ ]:
fud_harvester['new'] = fud_harvester.apply(lambda x: 200*x['Harvest Rate (FUD) / Day ']/(x['FUD'] + 2*x['FOMO'] + 4*x['ALPHA'] + 10*x['KEK'])
                                           , axis=1)

In [ ]:
fud_harvester.head(10)

In [ ]:
fud_reservoir['new'] = fud_reservoir.apply(lambda x: 2000/(x['FUD'] + 2*x['FOMO'] + 4*x['ALPHA'] + 10*x['KEK'])
                                           , axis=1)

In [ ]:
fud_reservoir.head(10)

Question: given time T, parcel P, alchemica type X, should you upgrade the harvestor/reservoir, or should you build multiple ones ? i.e. what is the best combination of harvestor/reservvoir of it ?

In [ ]:
"""


simplified version

- assume average number of alchemica in the parcel
- assume same number of harvestor/reservoir to be built for each type
- tell what level should we build for a given time T
- tell the result of each type of parcel humble, small, & spacious
- ignore maker as of now

"""

In [42]:
parcel_input = [1, 1, 1]


In [43]:
#total_parcels = sum(parcel_input)

In [44]:
class MyParcel_3:
    dim = None
    max_allowed_harvestors = [0,0,0,0]
    tag = None
    alchemica_content = None
    
    
    def __init__(self, dim):
        self.dim = dim
        
        if dim == (8,8):
            self.max_allowed_harvestor = [1,1,1,1]
            self.tag = 'humble'
            self.alchemica_content = [7077, 3539, 1769, 708]
        elif dim == (16,16):
            self.max_allowed_harvestor = [4,4,4,4]
            self.tag = 'reasonable'
            self.alchemica_content = [28308, 14154, 7077, 2831]
        elif dim == (32,64):
            self.max_allowed_harvestor = [32,32,32,32]
            self.tag = 'spacious'
            self.alchemica_content = [226465, 113233, 56616, 22647]
        
    def max_allowed_harvestors_for_alchemica(self, alchemica_type):
        if alchemica_type == 'FUD':
            return self.max_allowed_harvestor[0]
        elif alchemica_type == 'FOMO':
            return self.max_allowed_harvestor[1]
        elif alchemica_type == 'ALPHA':
            return self.max_allowed_harvestor[2]
        elif alchemica_type == 'KEK':
            return self.max_allowed_harvestor[3]
        else:
            return 0
        
    def get_alchemica_content(self, alchemica_type):
        if alchemica_type == 'FUD':
            return self.alchemica_content[0]
        elif alchemica_type == 'FOMO':
            return self.alchemica_content[1]
        elif alchemica_type == 'ALPHA':
            return self.alchemica_content[2]
        elif alchemica_type == 'KEK':
            return self.alchemica_content[3]
        else:
            return 0
        
    def print_parcel_info(self):
        print("parcel dim {}".format(self.dim))
        print("alchemica content ", self.alchemica_content)


In [45]:
Parcels = []

for i in range(parcel_input[2]):
    Parcels.append(MyParcel_3((32, 64)))

for i in range(parcel_input[1]):
    Parcels.append(MyParcel_3((16, 16)))

for i in range(parcel_input[0]):
    Parcels.append(MyParcel_3((8, 8)))

In [46]:
Parcels[1].print_parcel_info()

parcel dim (16, 16)
alchemica content  [28308, 14154, 7077, 2831]


In [47]:
# T = 200

# parcel_size = (8, 8)
# max_allowed_harvestor = 1
# num_fud = 1000

In [49]:
#num_harvestors_per_alchemica = 1


In [50]:
def compute_profit_for_time_T(harvestors_array, reservoirs_array, T, helper_obj, alchemica_factor, alchemica_amount):
    ## what should be the order in which harvestors and reservoirs are built, it is also important
    ## here we will first construct all reseervoirs, then all harvestors in desccending order
    
    days_passed = 0
    cost_till_now = 0
    harvest_till_now = 0
    
    for reservoir in reservoirs_array:
        cost_till_now += helper_obj.reservoir_cost_dict[reservoir]
        days_passed += helper_obj.reservoir_build_time[reservoir]
        
    #print("total reservoir cost -->", cost_till_now)
    
    for harvestor in sorted(harvestors_array, reverse=True):
        if harvestor == 0:
            continue
        
        cost_till_now += helper_obj.harvester_cost_dict[harvestor]
        days_passed += helper_obj.harvester_build_time[harvestor]
                
        harvest_till_now += alchemica_factor * helper_obj.harvester_rate[harvestor] * (T-days_passed) #TODO - also add all the time that has passed from level 1 to current upgrade
        
        
    if harvest_till_now > alchemica_amount:
        harvest_till_now = alchemica_amount
        
    #print("adding on harvestor cost -->", cost_till_now)
    #print("total harvest -->", harvest_till_now)
    
    return (harvest_till_now - cost_till_now)


In [ ]:
alchemica_type = 'KEK'
helper_obj = helper_dicts[alchemica_type]
alchemica_factor = alchemica_factor_dict[alchemica_type]
alchemica_amount = 708 * alchemica_factor

In [ ]:
compute_profit_for_time_T([1], [1], 100, helper_obj, alchemica_factor, alchemica_amount)

In [ ]:
# alchemica_type = 'KEK'
# reservoir_cost_dict = reservoir_cost_finder(alchemica_type)
# reservoir_build_time = reservoir_build_time_finder(alchemica_type)
# harvester_cost_dict = harvestor_cost_finder(alchemica_type)
# harvester_build_time = harvestor_build_time_finder(alchemica_type)
# harvester_rate = harvestor_harvestrate_finder(alchemica_type)
# alchemica_factor = alchemica_factor_dict[alchemica_type]

# # compute_profit_for_time_T([4, 0, 0, 0, 0], [1], 100, reservoir_cost_dict, reservoir_build_time, harvester_cost_dict,
# #                          harvester_build_time, harvester_rate, alchemica_factor)

In [ ]:
helper_obj.harvester_cost_dict

In [ ]:
helper_obj.reservoir_cost_dict

In [51]:
def compute_reservoir_needed(harvestors_array, helper_obj):
    ## keep on upgrading the current reservoir till highest level, then start building the 2nd one
    
    reservoirs_array = []
    
    total_daily_required_capacity = 0
    
    for harvestor in harvestors_array:
        if harvestor == 0:
            continue
        total_daily_required_capacity += helper_obj.harvester_rate[harvestor]
        
    
    #print("total daily required capacity -->", total_daily_required_capacity)
    
    current_capacity = 0
    while True:
        #print("loop")
        
        if total_daily_required_capacity > daily_reservoir_empty_frequency*helper_obj.reservoir_capacity[9]: #means more than 1 reservoir is needed
            reservoirs_array.append(9) #append maximum level
            total_daily_required_capacity -= daily_reservoir_empty_frequency*helper_obj.reservoir_capacity[9]
            
        else:
            for i in range(1, 10):
                if total_daily_required_capacity < daily_reservoir_empty_frequency*helper_obj.reservoir_capacity[i]:
                    reservoirs_array.append(i)
                    return reservoirs_array
            


In [ ]:
alchemica_type = 'KEK'

helper_obj = helper_dicts[alchemica_type]

compute_reservoir_needed([5, 0, 0, 0, 0], helper_obj)

In [52]:
def compute_harvestor_reservoir_for_parcel_alchemica(parcel, alchemica_type, T, parcel_alchemica_content):
    
    harvestors_built = [0]*parcel.max_allowed_harvestors_for_alchemica(alchemica_type)
    harvestors_built[0] = 1
    reservoirs_built = [1]
    
    helper_obj = helper_dicts[alchemica_type]

    alchemica_factor = alchemica_factor_dict[alchemica_type]
    
    alchemica_amount = parcel_alchemica_content * alchemica_factor
    
    total_spent = helper_obj.harvester_cost_dict[1] + helper_obj.reservoir_cost_dict[1] #building level 1 harvestor and reservoir

    original_harvestors_array = harvestors_built.copy()
    original_reservoirs_array = reservoirs_built.copy()

    while(True):
    
        optimized_harvestors_array = original_harvestors_array.copy()
        optimized_reservoirs_array = original_reservoirs_array.copy()


        optimized_profit = compute_profit_for_time_T(optimized_harvestors_array, optimized_reservoirs_array, T, helper_obj, alchemica_factor, alchemica_amount)
        # break the loop when current state is the max profit state

#         print("----------------------------------------")
#         print("new iteration begins")
#         print("current harvestor --> ", optimized_harvestors_array)
#         print("current reservoir --> ", optimized_reservoirs_array)
#         print("current profit -->", optimized_profit)


        # calculate next options
        for i in range(len(original_harvestors_array)):

            if original_harvestors_array[i] == 9:
                continue

            #print("\nupgrading harvestor number -->", i)

            #if I upgrade this harvestor, what is the marginal cost
            new_harvestors_array = original_harvestors_array.copy()
            new_harvestors_array[i] += 1

            #print("new harvestor --> ", new_harvestors_array)


            #compute the reservoir needed
            new_reservoirs_array = compute_reservoir_needed(new_harvestors_array, helper_obj)
            #print("new reservoir --> ", new_reservoirs_array)

            #print("alchemica factor {}, alchemica amount {}".format(alchemica_factor, alchemica_amount))


            #compute the total profit of the setup
            new_profit = compute_profit_for_time_T(new_harvestors_array, new_reservoirs_array, T, 
                                                   helper_obj, alchemica_factor, alchemica_amount)

            #print("new profit -->", new_profit)


            if new_profit > optimized_profit:
                optimized_harvestors_array = new_harvestors_array.copy()
                optimized_reservoirs_array = new_reservoirs_array.copy()
                optimized_profit = new_profit


        if original_harvestors_array == optimized_harvestors_array and original_reservoirs_array == optimized_reservoirs_array:
#             print("\n---------------------------------------------------------\n")
#             print("| optimal solution found\t\t\t\t|")

#             print("| optimal solution is -->", optimized_harvestors_array, optimized_reservoirs_array, "\t\t|")
#             print("| optimal profit -->", optimized_profit, "\t\t|")
#             print("\n---------------------------------------------------------\n")

            break

        else:
            original_harvestors_array = optimized_harvestors_array.copy()
            original_reservoirs_array = optimized_reservoirs_array.copy()

    return optimized_harvestors_array, optimized_reservoirs_array, optimized_profit
    


In [103]:
T = 10000
alchemica = 'KEK'
parcel = Parcels[0]
parcel.print_parcel_info()
parcel_alchemica_content = parcel.get_alchemica_content(alchemica)
compute_harvestor_reservoir_for_parcel_alchemica(Parcels[1], alchemica, T, parcel_alchemica_content)


parcel dim (32, 64)
alchemica content  [226465, 113233, 56616, 22647]


([4, 0, 0, 0], [3], 176323.02843413714)

In [93]:
#avg_alchemica_per_parcel_dict

{'humble': {'FUD': 7077, 'FOMO': 3539, 'ALPHA': 1769, 'KEK': 708},
 'reasonable': {'FUD': 28308, 'FOMO': 14154, 'ALPHA': 7077, 'KEK': 2831},
 'spacious': {'FUD': 226465, 'FOMO': 113233, 'ALPHA': 56616, 'KEK': 22647}}

In [53]:
#alchemica_factor_dict

{'FUD': 1.0,
 'FOMO': 2.7480987838280755,
 'ALPHA': 4.021756237158579,
 'KEK': 7.87385771813631}

In [119]:
T = 100
T

100

In [120]:
#Parcels[0].print_parcel_info(), Parcels[1].print_parcel_info(), Parcels[2].print_parcel_info()

In [121]:
result = []
for parcel in Parcels:
        
    pc = {}
    pc['parcel_size'] = " ".join([str(ele) for ele in list(parcel.dim)])
    
    pc['installations'] = {}
    total_profit_in_fud = 0
    
    #parcel.print_parcel_info()
    
    for alchemica in ['FUD', 'FOMO', 'ALPHA', 'KEK']:
        
        #print(parcel.print_parcel_info(), alchemica)
        
        parcel_alchemica_content = parcel.get_alchemica_content(alchemica)
        
        harvesters_array, reservoirs_array, profit = compute_harvestor_reservoir_for_parcel_alchemica(parcel, alchemica, T, parcel_alchemica_content)

            
        harvesters_array = list(filter(lambda num: num != 0, harvesters_array)) #removing extra zeros from harvester array
        
        #print("harvesters_array {}, reservoirs_array {}, profit_yield_in_fud {}".format(harvesters_array, reservoirs_array, int(profit)))

        if profit > 0:
            pc['installations'][alchemica + " " + "harvester"] = " ".join([str(ele) for ele in list(harvesters_array)])
            pc['installations'][alchemica + " " + "reservoir"] = " ".join([str(ele) for ele in list(reservoirs_array)])
            total_profit_in_fud += profit 
        
        
        #print("...........................................")
        #print("alchemica --> {}, profit --> {}".format(alchemica, profit))

    
    pc["profit_yield_in_fud"] = int(total_profit_in_fud)

    result.append(pc)
    #break

print(json.dumps(result, indent=4))
        

[
    {
        "parcel_size": "32 64",
        "installations": {
            "FUD harvester": "9 9",
            "FUD reservoir": "8",
            "FOMO harvester": "9",
            "FOMO reservoir": "6",
            "ALPHA harvester": "9 3",
            "ALPHA reservoir": "6",
            "KEK harvester": "9",
            "KEK reservoir": "7"
        },
        "profit_yield_in_fud": 134905
    },
    {
        "parcel_size": "16 16",
        "installations": {
            "FUD harvester": "9",
            "FUD reservoir": "7",
            "FOMO harvester": "9",
            "FOMO reservoir": "6",
            "ALPHA harvester": "9",
            "ALPHA reservoir": "6",
            "KEK harvester": "9",
            "KEK reservoir": "7"
        },
        "profit_yield_in_fud": 69097
    },
    {
        "parcel_size": "8 8",
        "installations": {
            "FUD harvester": "6",
            "FUD reservoir": "4",
            "FOMO harvester": "6",
            "FOMO reservoir": "4"